# Расскадровка видео: создание последовательностей изображений

За основу взят код из статьи:
+ Как с помощью Python извлекать кадры из видео 
https://waksoft.susu.ru/2021/11/16/kak-s-pomoshhyu-python-izvlekat-kadry-iz-video/
### Алгоритм
#### Вручную:
+ Скачиваем с Youtube видео-файлы и сохраняем их в папку `FromYoutube`
+ Для скачки используем сервис `https://ru.savefrom.net/`
+ Скачиваем `mp4` в наилучшем доступном качестве (720 и выше)

#### Код:
+ Импорт библиотек: Open CV, os, pandas, datetime.timedelta и datetime.data
+ Создается список видеофайлов в папке `FromYoutube`: используется библиотека `os`

### Импорт библиотек

In [13]:
import cv2 # Open CV - библиотека для обработки видео 
import os # библиотека для работы с операционной системой

import pandas as pd # для формирования списка кадров и сохранения его в excel-файлах
import numpy as np

from datetime import timedelta, date # для обработки меток времени
date_ =  pd.Timestamp.now().date()

### Создаем список видеофайлов в папке `FromYoutube_mp4`

In [14]:
from_youtube_folder = 'FromYoutube_mp4'
files_from_Youtube = os.listdir(from_youtube_folder)
files_from_Youtube

['Baryshnikov.mp4',
 'Black Swan.mp4',
 'Don Khihot.mp4',
 'Dying Swan.mp4',
 'Giselle.mp4',
 'Odetta.mp4',
 'Pavlova.mp4',
 'Queen of Hearts.mp4',
 'Sleeping Beauty.mp4',
 'Sugar Plum.mp4',
 'Tanec Fei Draze.mp4']

### Папка, в которую будут сохраняться изображения

In [15]:
folder_FromYoutube_jpeg = 'FromYoutube_jpeg'

### Функция `format_timedelta`
Для получения понятных меток времени конкретного кадра.
Эти метки используем при присваивании имен кадрам.

Например, пусть у конкретного кадра метка времени **237.88** секунда (т.е. `frame_duration=237.88`).
+ тогда с помощью функции `timedelta` преобразуем 237.88 в **datetime.timedelta(seconds=237, microseconds=880000)** 
    + (строчка `timedelta(seconds=frame_duration)`)

+ подаем `td = datetime.timedelta(seconds=237, microseconds=880000)` на вход **format_timedelta**
+ получаем красивое значение: **0-03-57.88**

In [16]:
def format_timedelta(td):
    """Служебная функция для классного форматирования объекта timedelta (например, 00:00:20.05)
    исключая микросекунды и сохраняя миллисекунды"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except:
        result, ms = (result + ".00").replace(":", "-").split(".")
        
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")

### Функция `get_saving_frames_durations`
+ Функция берет длительность видеофайла в секундах
+ Берет количество количество кадров в 1 секунде (25 кадров)
+ и возвращает список длительностей, в которые следует сохранять кадры

Так, для 1-ой секунды видео длительности кадров, которые следует сохранять, будут такими:
`[0.0, 0.04, 0.08, 0.12, 0.16, 0.2, 0.24, 0.28, 0.32, 0.36, 0.4, 0.44, 0.48, 0.52, 0.56, 0.6, 0.64, 0.68, 0.72, 0.76, 0.8, 0.84, 0.88, 0.92, 0.96]` 

In [17]:
def get_saving_frames_durations(cap, saving_fps):
    """функция, которая возвращает список длительностей, в которые следует сохранять кадры."""
    s = []
    
    # получаем продолжительность клипа, разделив количество кадров на количество кадров в секунду

    try:
        clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    except:
        clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / 29.97
    print(f'clip duration (sec): {clip_duration}') 

    
    for i in np.arange (0, clip_duration, 1/saving_fps):
        i = round(i,2)
        s.append(i)
    return s
 

#### Пример работы функции `get_saving_frames_durations`

In [18]:
# формируем полное имя видеофайла
filename = 'Baryshnikov.mp4'
video_file = os.path.join(from_youtube_folder, filename)
video_file

'FromYoutube_mp4\\Baryshnikov.mp4'

In [19]:
# создаем объект `cap`
cap = cv2.VideoCapture(video_file)
cap

< cv2.VideoCapture 0000019F3B1BE9B0>

In [20]:
# применяем нашу функцию
# для экономии места выводим только первые 13 (почти полсекнуды) значений из списка, которые возвращает функция
get_saving_frames_durations(cap, 25)[:13]


clip duration (sec): 81.28


[0.0, 0.04, 0.08, 0.12, 0.16, 0.2, 0.24, 0.28, 0.32, 0.36, 0.4, 0.44, 0.48]

### Функция `main`

In [21]:
SAVING_FRAMES_PER_SECOND = 25
def main(file):
    
    # формируем полное имя видеофайла для расскадровки:
    # сшиваем адрес папки и имя файла
    video_file = os.path.join(from_youtube_folder, file)
    filename, _ = os.path.splitext(file)
    print(f'Начинаю раскадровку файла {file}')

    # читать видео файл с помощью библиотеки Open CV
    cap = cv2.VideoCapture(video_file)
    
    # получить FSP видео
    # FSP - frames per second
    fps = cap.get(cv2.CAP_PROP_FPS)

    # заметила, что если у видеофайла FSP составляет 29.97 кадров в секунду, 
    # то `cap.get` возвращает ноль
    # для таких случаев принудительно заменяю 0 на 29.97
    if fps == 0:
        fps = 29.97
   
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    
    # получить список длительностей для сохранения
    saving_frames_durations = get_saving_frames_durations(cap,
                                                          saving_frames_per_second)

    # создать датасет на названиями кадров
    df = pd.DataFrame(columns=['frame'])
    
    # запускаем цикл
    count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            # выйти из цикла, если нет фреймов для чтения
            print('нет фреймов для чтения, заканчиваю расскадровку видео')
            break
            
        # получаем продолжительность, разделив количество кадров на FSP
        frame_duration = count / fps
        
        try:
            # получить самую раннюю продолжительность для сохранения
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # список пуск, все кадры длительности сохранены
            print('список пуск, все кадры длительности сохранены')
            break

        if frame_duration >= closest_duration:
            # если ближайшая длительность меньше или равна длительности кадра,
            # затем сохраняем фрейм
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            
            cv2.imwrite(os.path.join(folder_FromYoutube_jpeg,f"{filename}_{frame_duration_formatted}.jpg"), 
                        frame)
            
            # создать датасет на названиями кадров
            df.loc[len(df)] = f'{filename}_{frame_duration_formatted}'
            
            # удалить точку продолжительности из списка, так как эта точка длительности уже сохранена
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # увеличить количество кадров
        count += 1
    
    # сохраняю датафрейм со списком кадров
    df.to_excel(f'excel_seqeunces/{filename}.xlsx')

### Применяю функцию `main` ко всем видеофайлам в папке `FromYoutube`

In [ ]:
for file in files_from_Youtube:
    main(file)
print('Раскадровка завершена')